<a href="https://colab.research.google.com/github/RajiRai/GenAI-Tutorial/blob/main/GenAI_July18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install cohere weaviate-client==4.5.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.5/178.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import weaviate
import cohere

In [3]:
from google.colab import userdata
KEY=userdata.get('Cohere_key')
co = cohere.Client(KEY)


In [4]:
#connect weaviate db containing more than 10 million wiki vectors

auto_config = weaviate.auth.AuthApiKey(api_key="76320a90-53d8-42bc-b41d-678647c6672e")
client=weaviate.Client(
    url="https://cohere-demo.weaviate.network/",
    auth_client_secret=auto_config,
    additional_headers={
        "X-Cohere-Api-Key": KEY
    }
)

client.is_ready()

True

In [10]:
# Keyword search

def keyword_search(query, results_lang="en", num_results=10):
  properties=["text", "title", "url", "views", "lang", "_additional {distance}"]

  where_filter={
      "path": ["lang"],
      "operator": "Equal",
      "valueString": results_lang
  }

  response = (
      client.query.get("Articles", properties)
      .with_bm25(
          query=query,
      )
      .with_where(where_filter)
      .with_limit(num_results)
      .do()
  )

  result = response["data"]["Get"]["Articles"]

  return result


In [7]:
def print_result(result):
  for item in result:
    print(item["title"])
    print(item['views'])
    print(item["text"])
    print(item["url"])
    print()

In [15]:
simple_query = "Who invented Python programming language?"
keyword_search_result = keyword_search(simple_query)
print_result(keyword_search_result)

Python (programming language)
3000
Since 2003, Python has consistently ranked in the top ten most popular programming languages in the TIOBE Programming Community Index where, , it is the most popular language (ahead of Java, and C). It was selected Programming Language of the Year (for "the highest rise in ratings in a year") in 2007, 2010, 2018, and 2020 (the only language to do so four times).
https://en.wikipedia.org/wiki?curid=23862

Python (programming language)
3000
A common neologism in the Python community is "pythonic", which has a wide range of meanings related to program style. "Pythonic" code may use Python idioms well, be natural or show fluency in the language, or conform with Python's minimalist philosophy and emphasis on readability. Code that is difficult to understand or reads like a rough transcription from another programming language is called "unpythonic".
https://en.wikipedia.org/wiki?curid=23862

Python (programming language)
3000
Python is a multi-paradigm pro

In [19]:
another_query = "Is eat apple a day helpful?"
keyword_search_result = keyword_search(another_query)
print_result(keyword_search_result)

Quince
2000
Some varieties of quince, such as 'Aromatnaya' and 'Kuganskaya' do not require cooking and can be eaten raw. However, most varieties of quince are too hard, astringent and sour to eat raw unless "bletted" (softened by frost and subsequent decay). High in pectin, they are used to make jam, jelly and quince pudding, or they may be peeled, then roasted, baked or stewed; pectin levels diminish as the fruit ripens. Long cooking with sugar turns the flesh of the fruit red by formation of anthocyanins. The very strong perfume means they can be added in small quantities to apple pies and jam to enhance the flavor. Adding a diced quince to apple sauce will enhance the taste of the apple sauce with the chunks of relatively firm, tart quince. The term "marmalade", originally meaning a quince jam, derives from "marmelo", the Portuguese word for this fruit.
https://en.wikipedia.org/wiki?curid=294752

Uber Eats
2000
In 2019, Uber Eats said it would deliver food to customers by drones fro

In [20]:
# Dense Retrieval

def dense_retrieval(query, results_lang="en", num_results=10):
  nearText = {"concepts": [query]}
  properties=["text", "title", "url", "views", "lang", "_additional {distance}"]
  where_filter={
      "path": ["lang"],
      "operator": "Equal",
      "valueString": results_lang
  }

  response = (
      client.query.get("Articles", properties)
      .with_near_text(nearText)
      .with_where(where_filter)
      .with_limit(num_results)
      .do()
  )

  result = response["data"]["Get"]["Articles"]

  return result



In [22]:
simple_query = "Who invented Python?"
dense_retrieval_result = dense_retrieval(simple_query)
print_result(dense_retrieval_result)

Python (programming language)
3000
Python was conceived in the late 1980s by Guido van Rossum at Centrum Wiskunde & Informatica (CWI) in the Netherlands as a successor to the ABC programming language, which was inspired by SETL, capable of exception handling (from the start plus new capabilities in Python 3.11) and interfacing with the Amoeba operating system. Its implementation began in December 1989. Van Rossum shouldered sole responsibility for the project, as the lead developer, until 12 July 2018, when he announced his "permanent vacation" from his responsibilities as Python's "benevolent dictator for life", a title the Python community bestowed upon him to reflect his long-term commitment as the project's chief decision-maker. In January 2019, active Python core developers elected a five-member Steering Council to lead the project.
https://en.wikipedia.org/wiki?curid=23862

Python (programming language)
3000
Guido van Rossum began working on Python in the late 1980s as a successo

In [23]:
another_query = "Is eating a apple a day helpful?"
keyword_search_result = keyword_search(another_query)
print_result(keyword_search_result)

Asthma
2000
Evidence is insufficient to support the usage of vitamin C or vitamin E for controlling asthma. There is tentative support for use of vitamin C in exercise induced bronchospasm. Fish oil dietary supplements (marine n-3 fatty acids) and reducing dietary sodium do not appear to help improve asthma control. In people with mild to moderate asthma, treatment with vitamin D supplementation may reduce the risk of asthma exacerbations, however, it is not clear if this is only helpful for people who have low vitamin D levels to begin with (low baseline vitamin D). There is no strong evidence to suggest that vitamin D supplements improve day-to-day asthma symptoms or a person's lung function. There is no strong evidence to suggest that adults with asthma should avoid foods that contain monosodium glutamate (MSG). There have not been enough high-quality studies performed to determine if children with asthma should avoid eating food that contains MSG.
https://en.wikipedia.org/wiki?curi

In [24]:
arabic_results = dense_retrieval (another_query, results_lang="ar")
keyword_search_result = keyword_search(another_query)
print_result(arabic_results)

منجا (فاكهة)
900
هل تعلم أن المانجو غنية بفيتامين A. كوب واحد من شرائح المانجو يعطيك 25٪ من كمية الحاجة اليومية من فيتامين A. المانجو تساعد في العناية بالعين فتتمتّع ببصر جيد، وتحارب جفاف العين، وكذلك تمنع العمى الليلي.
https://ar.wikipedia.org/wiki?curid=25718

عنب
800
- توجد في العنب كمية جيدة من الألياف الغذائية المفيدة في منع حدوث الإمساك، وفي إنقاص مستوى الكوليسترول في الدم، وفي تنظيم مستوى السكر، وفي المساعدة على التخلص من المواد والمخلفات الضارة.
https://ar.wikipedia.org/wiki?curid=7796

زبيب
500
تشير البيانات المقدمة في الدورة العلمية السنوية للكلية الأمريكية لأمراض القلب الحادية والستين في عام 2012 إلى أنه عند الأفراد الذين يعانون من ارتفاع طفيف في ضغط الدم، قد يؤدي الاستهلاك الروتيني للزبيب (ثلاث مرات في اليوم) إلى خفض ضغط الدم بشكل ملحوظ، مقارنةً بتناول أنواع أخرى شائعة من الوجبات الخفيفة.
https://ar.wikipedia.org/wiki?curid=156422

أفوكادو
900
كما تحتوي نصف ثمرة أفوكادو كبيرة على ضعف كمية [[ألياف غذائية|الألياف الغذائية]] الموجودة في تفاحتين، والمعروف أن الألياف الغذائية مه

In [26]:
spanish_results = dense_retrieval (another_query, results_lang="es")
keyword_search_result = keyword_search(another_query)
print_result(spanish_results)

Manzana
1000
El proverbio, "Una manzana al día mantiene alejado al médico", abordando los supuestos beneficios para la salud de la fruta, se remonta al Gales del , cuando la frase original era "Come una manzana al ir a la cama, y evitarás que el médico se gane el pan". En el y principios del , la frase evolucionó a "una manzana al día, no hay médico que pagar" y "una manzana al día envía al médico lejos"; el fraseo ahora comúnmente utilizado se grabó por primera vez en 1922. A pesar del proverbio, no hay evidencia de que comer una manzana diariamente tenga efectos significativos en la salud.
https://es.wikipedia.org/wiki?curid=50309

Obesidad
1000
El estudio "Women's Health Initiative Randomized Controlled Dietary Modification Trial" encontró que una dieta con un total de 20 % de la energía proveniente de la energía y un incremento en el consumo de vegetales y frutas hasta de al menos 5raciones diarias y granos de hasta al menos 6raciones diarias resultó en: no hubo reducción en la enf

In [29]:
spanish_query = "La manzana cruda actúa como un excelente dentífrico por dos razones"
french_results = dense_retrieval (another_query, results_lang="fr")
keyword_search_result = keyword_search(spanish_query)
print_result(french_results)

Fruit (alimentation humaine)
700
Les fruits évitent le surpoids. Avec une moyenne de 50 kcal pour , les fruits sont peu caloriques tout en étant rassasiants, grâce aux fibres alimentaires qu'ils contiennent, par exemple les pommes. Ils constituent aussi une mine de vitamines et de sels minéraux. Ils tiennent une place de choix dans tous les menus équilibrés afin de lutter contre la surcharge pondérale et l'obésité.
https://fr.wikipedia.org/wiki?curid=5490

Pomme
1000
L'apport énergétique de la pomme (/, soit 85 kcal pour une pomme de taille moyenne) provient non pas de graisses, mais de fructose et de glucides assimilables lentement par l'organisme. Le profil nutritionnel de la pomme en fait un fruit tout à fait adapté aux sportifs. En effet, dans le cadre d'activités physiques, les composantes de la pomme agissent de façon bénéfique sur l'organisme et ce, avant, pendant et après l'effort.
https://fr.wikipedia.org/wiki?curid=5461

Avocat (fruit)
1000
La consommation d'avocats a un effe

In [30]:
# Rerank

def rerank_responses(query, responses, num_responses=3):
  reranked_responses = co.rerank(
      query=query,
      documents=responses,
      top_n=num_responses,
      model="rerank-english-v3.0",
      return_documents = True
  )

  return reranked_responses

In [33]:
query = "who was the first person to reach moon?"

keyword_searches_to_improve = keyword_search(query, num_results=100)


In [34]:
for r in keyword_searches_to_improve:
  print(r["title"],':',r["text"])

First-person shooter : There are many free-to-play first-person shooters on the market now, including "", "Apex Legends", "Team Fortress 2", "Planetside 2, and Halo Infinite Multiplayer." Some games are released as free-to-play as their intended business model and can be highly profitable ("League of Legends" earned $2 billion in 2017), but others such as "Eternal Crusade" begin their life as paid games and become free-to-play later to reach a wider audience after an initially disappointing reception. Some player communities complain about freemium first-person-shooters, fearing that they create unbalanced games, but many game designers have tweaked prices in response to criticism, and players can usually get the same benefits by playing longer rather than paying.
Ban Ki-moon : Ban took the first foreign trip of his term to attend the African Union summit in Addis Ababa, Ethiopia, in January 2007 as part of an effort to reach out to the Group of 77. He repeatedly identified Darfur as t

In [35]:
reranked_keyword_responses = rerank_responses(query, keyword_searches_to_improve, num_responses=3)

In [37]:
for idx, r in enumerate(reranked_keyword_responses.results):
  print(f"Document Rank :{idx+1}, Document index : {r.index}")
  print(f"Title: {r.document.title}")
  print(f"URL: {r.document.url}")
  print(f"Document : {r.document.text}")
  print(f"Relevence Score: {r.relevance_score: .2f}")
  print("\n\n")

Document Rank :1, Document index : 21
Title: Moon
URL: https://en.wikipedia.org/wiki?curid=19331
Document : The first artificial object to reach the Moon was the Soviet Union's Luna 2 uncrewed spacecraft in 1959; this was followed by the first successful soft landing by Luna 9 in 1966. The only human lunar missions to date have been those of the United States' Apollo program, which landed twelve men on the surface between 1969 and 1972. These and later uncrewed missions returned lunar rocks that have been used to develop a detailed geological understanding of the Moon's origins, internal structure, and subsequent history.
Relevence Score:  0.98



Document Rank :2, Document index : 16
Title: Alan Shepard
URL: https://en.wikipedia.org/wiki?curid=63727
Document : Faced with intense competition from the other astronauts, particularly John Glenn, Shepard quit smoking and adopted Glenn's habit of taking a morning jog. On January 19, 1961, Gilruth informed the seven astronauts that Shepard h

In [38]:
# Generative models with dense retrieval

query = "Who was first person to reach the moon?"

prediction_without_search = [
    co.chat(
        message=query,
        max_tokens=50,

    )for _ in range(5)

]

In [39]:
for p in prediction_without_search:
  print(p.text)

Neil Armstrong was the first person to walk on the Moon. On July 20, 1969, Armstrong, an American astronaut, made history by taking the first human steps on the lunar surface as part of the Apollo 11
Neil Armstrong was the first person to walk on the Moon. On July 20, 1969, Armstrong, an American astronaut, made history by taking humanity's first small step on the lunar surface, an event that was broadcast live
Neil Armstrong was the first person to walk on the Moon. On July 20, 1969, Armstrong, an American astronaut, made history by taking humanity's first small step on the lunar surface, an event that was broadcast live
Neil Armstrong was the first person to walk on the Moon. On July 20, 1969, Armstrong, an American astronaut, made history by taking the first human steps on the lunar surface as part of the Apollo 11
Neil Armstrong was the first person to walk on the Moon. On July 20, 1969, Armstrong, an American astronaut, made history by taking humanity's first small step on the lun

In [41]:
# use dense retrieval

responses=dense_retrieval(query,num_results=20)
print(responses)
print_result(responses)

[{'_additional': {'distance': -148.52777}, 'lang': 'en', 'text': "The first artificial object to reach the Moon was the Soviet Union's Luna 2 uncrewed spacecraft in 1959; this was followed by the first successful soft landing by Luna 9 in 1966. The only human lunar missions to date have been those of the United States' Apollo program, which landed twelve men on the surface between 1969 and 1972. These and later uncrewed missions returned lunar rocks that have been used to develop a detailed geological understanding of the Moon's origins, internal structure, and subsequent history.", 'title': 'Moon', 'url': 'https://en.wikipedia.org/wiki?curid=19331', 'views': 3000}, {'_additional': {'distance': -148.07477}, 'lang': 'en', 'text': 'Apollo 11 (July 16–24, 1969) was the American spaceflight that first landed humans on the Moon. Commander Neil Armstrong and lunar module pilot Buzz Aldrin landed the Apollo Lunar Module "Eagle" on July 20, 1969, at 20:17 UTC, and Armstrong became the first pe

In [42]:
context = [r["text"] for r in responses]
context[:10]

["The first artificial object to reach the Moon was the Soviet Union's Luna 2 uncrewed spacecraft in 1959; this was followed by the first successful soft landing by Luna 9 in 1966. The only human lunar missions to date have been those of the United States' Apollo program, which landed twelve men on the surface between 1969 and 1972. These and later uncrewed missions returned lunar rocks that have been used to develop a detailed geological understanding of the Moon's origins, internal structure, and subsequent history.",
 'Apollo 11 (July 16–24, 1969) was the American spaceflight that first landed humans on the Moon. Commander Neil Armstrong and lunar module pilot Buzz Aldrin landed the Apollo Lunar Module "Eagle" on July 20, 1969, at 20:17 UTC, and Armstrong became the first person to step onto the Moon\'s surface six hours and 39 minutes later, on July 21 at 02:56 UTC. Aldrin joined him 19 minutes later, and they spent about two and a quarter hours together exploring the site they had

In [43]:
prompt=f"""
Use the information provide below to answer the questions in the end. If the answer to the question is not contained in the information, say "The answer is not in the context"

Context Information: {context}

Question: {query}

"""


In [44]:
prediction_with_search = [
    co.chat(
        message=prompt,
        max_tokens=50,
    )for _ in range(5)

]


In [45]:
for p in prediction_with_search:
  print(p.text)

The first person to reach the Moon was Neil Armstrong, commander of the Apollo 11 mission. He set foot on the lunar surface on July 21, 1969, at 02:56 UTC.
The first person to reach and walk on the Moon was Neil Armstrong, commander of the Apollo 11 mission, on July 21, 1969.
The first person to reach and walk on the Moon was Neil Armstrong, commander of the Apollo 11 mission, on July 21, 1969.
The first person to reach the Moon was Neil Armstrong, commander of the Apollo 11 mission. He set foot on the Moon on July 21, 1969, at 02:56 UTC.
The first person to reach and walk on the Moon was Neil Armstrong, commander of the Apollo 11 mission, on July 21, 1969, at 02:56 UTC.


In [53]:
! pip install cohere jsonlines

In [54]:
import os
import jsonlines
import json
import cohere

In [49]:
# get the subset of the dataset

phrase = "coherent"

dataset_list = []

with jsonlines.open("/content/drive/MyDrive/GENAI-RAW/train.jsonl") as f:
  for line in f.iter():
    if phrase in line["src"].split(":")[0]:
      dataset_list.append(line)


In [50]:
print(dataset_list)

[{'_id': '57220', 'task': 'coherence', 'src': "Make the text coherent: The Bank's main strategy is to further expand its network and increase its lending activities with particular focus on the SME sector. The EBRD helps Bank, by developing and financing Bank's portfolio of and strengthening the bank's funding base.", 'tgt': "The Bank's main strategy is to further expand its network and increase its lending activities with particular focus on the SME sector. The EBRD helps Union Bank, by developing and financing its portfolio of and strengthening the bank's funding base."}, {'_id': '57225', 'task': 'coherence', 'src': 'Make the text coherent: It was not illegal under international law ; captured foreign sailors were released. Confederates went to prison camps.', 'tgt': 'It was not illegal under international law ; captured foreign sailors were released, while Confederates went to prison camps.'}, {'_id': '57229', 'task': 'coherence', 'src': "Make the text coherent: The Union blockade w

In [51]:
# split data into train and test set

dataset_list_train = dataset_list[:800]
dataset_list_test = dataset_list[800:]

print ("Total number of samples:", len(dataset_list))
print ("Total number of train samples:", len(dataset_list_train))
print ("Total number of test samples:", len(dataset_list_test))

Total number of samples: 927
Total number of train samples: 800
Total number of test samples: 127


In [55]:
for item in dataset_list_train[:10]:
  print(item["src"])
  print(item["tgt"])
  print("--"* 100)

Make the text coherent: The Bank's main strategy is to further expand its network and increase its lending activities with particular focus on the SME sector. The EBRD helps Bank, by developing and financing Bank's portfolio of and strengthening the bank's funding base.
The Bank's main strategy is to further expand its network and increase its lending activities with particular focus on the SME sector. The EBRD helps Union Bank, by developing and financing its portfolio of and strengthening the bank's funding base.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Make the text coherent: It was not illegal under international law ; captured foreign sailors were released. Confederates went to prison camps.
It was not illegal under international law ; captured foreign sailors were released, while Confederates went to prison camps.
--------

In [57]:
# arrange the data to suit cohere format

def create_chat_ft_data(preamble, user_message, chatbot_message) :
  formatted_data = {
      "message" : [
          {
              "role" : "System",
              "content" : preamble
          },
          {
              "role" : "User",
              "content" : user_message
          },
          {
              "role" : "Chatbot",
              "content" : chatbot_message
          }
      ]
  }
  return formatted_data

In [58]:
preamble = "You are a writing assistant"

In [76]:
# create jsonl file out of training data

def create_jsonl_from_list(file_name, dataset_segment, preamble):
  path=f"{file_name}.jsonl"
  if not os.path.exists(path) :
    with open(path, "w+") as file :
      for item in dataset_segment:
        user_message = item["src"]
        chatbot_message = item["tgt"]
        formatted_data = create_chat_ft_data(preamble, user_message, chatbot_message)
        file.write(json.dumps(formatted_data) + "\n")
      file.close()

In [77]:
file_name = "coedit_coherence_train"
create_jsonl_from_list(file_name, dataset_list_train, preamble)

In [78]:
with jsonlines.open(f"{file_name}.jsonl") as f:
  [print(line) for _, line in zip(range(3),f)]


{'message': [{'role': 'System', 'content': 'You are a writing assistant'}, {'role': 'User', 'content': "Make the text coherent: The Bank's main strategy is to further expand its network and increase its lending activities with particular focus on the SME sector. The EBRD helps Bank, by developing and financing Bank's portfolio of and strengthening the bank's funding base."}, {'role': 'Chatbot', 'content': "The Bank's main strategy is to further expand its network and increase its lending activities with particular focus on the SME sector. The EBRD helps Union Bank, by developing and financing its portfolio of and strengthening the bank's funding base."}]}
{'message': [{'role': 'System', 'content': 'You are a writing assistant'}, {'role': 'User', 'content': 'Make the text coherent: It was not illegal under international law ; captured foreign sailors were released. Confederates went to prison camps.'}, {'role': 'Chatbot', 'content': 'It was not illegal under international law ; captured

In [ ]:
#need finetune model key to run these codes

for item in dataset_list_test[:3]:
    # User prompt
    user_message = item["src"]
    # Desired/target response from dataset
    tgt_message = item["tgt"]

    # Get default model response
    response_pretrained=co.chat(
        message=user_message,
        preamble=preamble,
        )

    # Get fine-tuned model response
    response_finetuned = co.chat(
        message=user_message,
        model='acb944bb-fb49-4c29-a15b-e6a245a7bdf9-ft',
        preamble=preamble,
        )

    print(f"User: {user_message}","\n-----")
    print(f"Desired response: {tgt_message}","\n-----")
    print(f"Default model's response: {response_pretrained.text}","\n-----")
    print(f"Fine-tuned model's response: {response_finetuned.text}")


    print("-"*100,"\n\n")



In [ ]:
#need finetune model key to run these codes

# Create a conversation ID
import uuid
conversation_id = str(uuid.uuid4())

print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break

    # Chatbot response
    stream = co.chat_stream(message=message,
                              model='acb944bb-fb49-4c29-a15b-e6a245a7bdf9-ft',
                              preamble=preamble,
                              conversation_id=conversation_id)

    print("Chatbot: ", end='')

    for event in stream:
        if event.event_type == "text-generation":
            print(event.text, end='')

    print(f"\n{'-'*50}\n")

In [ ]:
# Langchain and cohere

In [79]:
! pip install langchain Cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.8 MB/s eta 0:00:00


In [80]:
import os

os.environ["COHERE_API_KEY"] = KEY

In [81]:
! pip install -U langchain-community langchain-cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.3 MB/s eta 0:00:00


In [84]:
from langchain_community.llms import Cohere

llm = Cohere(model="command", temperature=0.5)

text = "What are potentials frameworks that make LLM easy to use?"

print(llm.invoke(text))

 There are several potential frameworks that could make LLMs easier to use directly as a user, especially through more natural and intuitive interfaces:

1. **Natural Language Interface**: Developing a more human-like interface to interact with LLMs can make them more accessible and user-friendly. This can include methods of interaction like speech, or more intuitive and flexible command systems.

2. **Contextual Understanding**: LLMs that can understand the context and intent behind user inputs and queries, rather than just responding to individual commands, will provide more coherent and intuitive interactions. This would involve incorporating memory, reasoning, and predictive capabilities into LLMs.

3. **Generative Interface**: Rather than just responding to queries, LLMs could offer more sophisticated options for generating content and ideas. For example, users could provide broad prompts and specifications, and the LLM could generate a range of potential responses or creative pie

In [85]:
! pip install huggingface_hub


In [86]:
from getpass import getpass
from huggingface_hub import notebook_login

notebook_login()

In [88]:
HUGGINGFACEHUB_API_TOKEN = getpass()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

··········


In [99]:
from langchain_community.llms import huggingface_hub

repo_id = "google/flan-t5-base"
llm = huggingface_hub.HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0, "max_length":70})

text = "What are few good frameworks to use with LLMs"

print(llm.invoke(text))


a sys.sys.setrecursionlimit(10 kHz)


In [100]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 3.8 MB/s eta 0:00:00


In [101]:
# to connect to openAI need openAI api key to run

os.environ["OPENAI_API_KEY"] = ""

from langchain.llms import OpenAI

llm = OpenAI(temperature=0.5)

text = "What are few good frameworks to use with LLMs"

print(llm.invoke(text))



/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [103]:
from langchain_cohere import ChatCohere
from langchain_core.messages import (AIMessage, HumanMessage, SystemMessage)

chat = ChatCohere(model="command-light", temperature=0, max_tokens=100)

In [111]:
messages = [
    SystemMessage(content="You are a helpful assistant that translate"),
    HumanMessage(content="Translate the following english to english: 'I love programming'"),

]

chat(messages)

AIMessage(content='"I love programming" in English translates to "I love coding" or "I love computer programming".', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '21263afa-7b7e-4db5-8434-56ea84e8f82c', 'token_count': {'input_tokens': 84, 'output_tokens': 22}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '21263afa-7b7e-4db5-8434-56ea84e8f82c', 'token_count': {'input_tokens': 84, 'output_tokens': 22}}, id='run-57547594-dfe5-4778-923d-dd5e5cd1edd3-0', usage_metadata={'input_tokens': 84, 'output_tokens': 22, 'total_tokens': 106})

In [114]:
chat = ChatCohere(model="command-light", temperature=0, max_tokens=100)

context = [SystemMessage(content="You are a helpful assistant that like to have conversation")]

while True:
  user_message = input("User: ")
  context.append(HumanMessage(content=user_message))
  if user_message.lower() == "quit":
    break

  response = chat(context)
  context.append(AIMessage(content=response.content))
  print(f"Chatbot: {response.content}")


User: do you know karnataka
Chatbot: Yes, I do!  Karnataka is a state in India and it is known for its beautiful landscapes, rich culture, and many popular tourist attractions.  I can provide information and various topics related to Karnataka.
User: is Hampi located in karnataka
Chatbot: Yes, Hampi is a historic city and UNESCO World Heritage Site located in the northern part of Karnataka, India. particular. It is famous for its particular ancient and medieval era due to its impressive monuments and temples.
User: quit


In [ ]:
llm = ChatCohere(
    model="command-r-plus",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=3
)


In [119]:
messages = [
    (
        "system",
        "you are a helpful assistant that translates English to French"
    ),
    (
        "human",
        "Translate the following English to French: 'I love programming'"
    )

]

In [121]:
ai_msg=llm.invoke(messages)
ai_msg

"'I love programming'"

In [128]:
# Chaining using prompt template and llm

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages (
    [
    ("system", "You are a helpful assistant that translate {input_language} to {output_language}"),
    ("human", "{input_text}")
    ]


)

llm = ChatCohere(model="command-light", temperature=0, max_tokens=None)

chain = prompt | llm

chain.invoke(
    {
        "input_text" : "Welcome to GenAI class",
        "input_language" : "English",
        "output_language" : "German",

    }
)


AIMessage(content='Hallo und Willkom!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1373564d-ff36-4da4-91cc-7e2c056d23b4', 'token_count': {'input_tokens': 80, 'output_tokens': 6}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '1373564d-ff36-4da4-91cc-7e2c056d23b4', 'token_count': {'input_tokens': 80, 'output_tokens': 6}}, id='run-836c3b31-92a5-4a9f-905f-17c8610537b1-0', usage_metadata={'input_tokens': 80, 'output_tokens': 6, 'total_tokens': 86})

In [129]:
# Langchain Embeddings

from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-english-light-v2.0")
text = "This is a test document."

query_result = embeddings.embed_query(text)

print (query_result)

[0.46606445, -0.07244873, 1.7421875, 0.10284424, -0.54296875, 0.5703125, -0.3557129, -0.23779297, 1.7197266, 0.65966797, -1.6201172, 1.0224609, 0.1083374, 0.9472656, 0.9921875, 0.9663086, 0.55615234, -0.67626953, 1.3544922, -0.33325195, -0.32836914, 1.6787109, 0.43286133, -0.10888672, 1.7216797, 1.2568359, 4.3632812, 0.24572754, 2.1289062, -1.8818359, 0.61572266, 0.5649414, -0.41088867, 1.0302734, 0.6933594, -0.27807617, 0.8823242, -0.30249023, 1.7783203, -0.61621094, 1.0615234, -0.9057617, 0.6772461, 0.88720703, -1.9296875, -0.8588867, 0.013999939, -1.3632812, 1.1298828, -1.3173828, 0.44799805, 0.11114502, 1.8359375, -0.1751709, 0.25634766, -0.23046875, -1.71875, -0.80322266, -0.14746094, 0.10369873, 0.97509766, -0.7807617, -0.6230469, -1.4921875, -0.5410156, -0.20495605, 1.7275391, -0.097351074, -0.6855469, 1.5205078, 1.0166016, 1.5302734, -0.5371094, 0.20397949, 2.1542969, -3.625, -1.9580078, -0.03994751, 0.097839355, -2.84375, 1.078125, 1.9677734, 0.47436523, -1.6748047, 0.31103516

In [131]:
docs_result = embeddings.embed_documents([text,text])

print (docs_result)

[[0.46606445, -0.07244873, 1.7421875, 0.10284424, -0.54296875, 0.5703125, -0.3557129, -0.23779297, 1.7197266, 0.65966797, -1.6201172, 1.0224609, 0.1083374, 0.9472656, 0.9921875, 0.9663086, 0.55615234, -0.67626953, 1.3544922, -0.33325195, -0.32836914, 1.6787109, 0.43286133, -0.10888672, 1.7216797, 1.2568359, 4.3632812, 0.24572754, 2.1289062, -1.8818359, 0.61572266, 0.5649414, -0.41088867, 1.0302734, 0.6933594, -0.27807617, 0.8823242, -0.30249023, 1.7783203, -0.61621094, 1.0615234, -0.9057617, 0.6772461, 0.88720703, -1.9296875, -0.8588867, 0.013999939, -1.3632812, 1.1298828, -1.3173828, 0.44799805, 0.11114502, 1.8359375, -0.1751709, 0.25634766, -0.23046875, -1.71875, -0.80322266, -0.14746094, 0.10369873, 0.97509766, -0.7807617, -0.6230469, -1.4921875, -0.5410156, -0.20495605, 1.7275391, -0.097351074, -0.6855469, 1.5205078, 1.0166016, 1.5302734, -0.5371094, 0.20397949, 2.1542969, -3.625, -1.9580078, -0.03994751, 0.097839355, -2.84375, 1.078125, 1.9677734, 0.47436523, -1.6748047, 0.3110351